In [1]:
using JuMP
using Ipopt

In [72]:
num_nodes = 20

R = 0:num_nodes #Rows and columns
N = 1:(num_nodes+1)^2

IR=IC=JR=JC=R
I=J=K=N

loc = Dict([i*length(R)+j+1=>(i,j) for i∈R,j∈R])

start = Dict(i=>0 for i∈N)
start[421]=1

ending = Dict(i=>0 for i∈N)
ending[21]=1


diag = 4

arcs = []
dist = Dict()

W = length(R)-1

#Build the grid
for (n1,(r1,c1))∈loc,(n2,(r2,c2))∈loc
    if max(abs(r1-r2),abs(c1-c2))<=diag & min(abs(r1-r2),abs(c1-c2))==1
        push!(arcs,(n1,n2))
        dist[n1,n2] = sqrt((r1-r2)^2+(c1-c2)^2)/W
    end

    if max(abs(r1-r2),abs(c1-c2))==1
        push!(arcs,(n1,n2))
        dist[n1,n2] = sqrt((r1-r2)^2+(c1-c2)^2)/W
    end
end


H = [:h0] #Hazards
hloc = Dict(:h0 => (0,0)) #hazard locations

pn = Dict() #probablity of success at each node
for (n,(r,c)) ∈ loc
    pn[n] = prod(sqrt((r-r1)^2+(c-c1)^2)/sqrt(2*W^2) for (h,(r1,c1))∈hloc)
end


p = Dict() #Probability of success on each arc
for (i,j) in arcs
    p[i,j] = .5 * (pn[i]+pn[j])
end

λ = 1 #weight on speed

1;

In [73]:
routechoice = Model(Ipopt.Optimizer)

@variable(routechoice,X[N,N]>=0)

@objective(routechoice,Min, sum(X[r,c]*dist[r,c]*(λ-(1-λ)*log(p[r,c])) for (r,c)∈arcs))

@constraint(routechoice, conservation[k=N],
    sum(X[r,k] for (r,c)∈arcs if c==k) + start[k] == sum(X[k,c] for (r,c)∈arcs if r==k) + ending[k]
)

λ=.5

optimize!(routechoice)


This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.0.

Number of nonzeros in equality constraint Jacobian...:     6560
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:   194481
                     variables with only lower bounds:   194481
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      441
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.9713689e+00 1.00e+00 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [76]:
value.(X)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:441
    Dimension 2, 1:441
And data, a 441×441 Matrix{Float64}:
 7652.66           9.91929e-9  7652.66        …  7652.66        7652.66
    6.72313e-9  7652.66           1.27206e-8     7652.66        7652.66
 7652.66           5.15448e-9  7652.66           7652.66        7652.66
 7652.66        7652.66           3.93458e-9     7652.66        7652.66
 7652.66        7652.66        7652.66           7652.66        7652.66
 7652.66        7652.66        7652.66        …  7652.66        7652.66
 7652.66        7652.66        7652.66           7652.66        7652.66
 7652.66        7652.66        7652.66           7652.66        7652.66
 7652.66        7652.66        7652.66           7652.66        7652.66
 7652.66        7652.66        7652.66           7652.66        7652.66
    ⋮                                         ⋱                    ⋮
 7652.66        7652.66        7652.66           7652.66        765

In [79]:
dist[24,1]

KeyError: KeyError: key (24, 1) not found

In [81]:
[a for (a,b) in arcs if b==1]

3-element Vector{Int64}:
 23
 22
  2